In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from PIL import Image
import os


dataset_path_cat = '/content/drive/MyDrive/cats_dogs/cat'
dataset_path_dogs = '/content/drive/MyDrive/cats_dogs/dog'
#the output size
output_size = (256, 256)

# resize the images
for filename in os.listdir(dataset_path_cat):
    if filename.endswith('.jpg'):
        image_path = os.path.join(dataset_path_cat, filename)
        image = Image.open(image_path)
        image = image.resize(output_size)
        image.save(image_path)

for filename in os.listdir(dataset_path_dogs):
    if filename.endswith('.jpg'):
        image_path = os.path.join(dataset_path_dogs, filename)
        image = Image.open(image_path)
        image = image.resize(output_size)
        image.save(image_path)
        

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input


dataset_path = '/content/drive/MyDrive/cats_dogs'

#the output size
output_size = (224, 224)

# Load the ResNet50V2 CNN
model = ResNet50V2(weights='imagenet', include_top=False)

#function to extract features from an image
def extract_features(image_path):
    img = load_img(image_path, target_size=output_size)
    img = img_to_array(img)
    img = preprocess_input(img)
    features = model.predict(np.array([img]))
    features = features.flatten()
    return features

#extracting  the features of each image in the directories
features = []
labels = []
for label, folder in enumerate(['cat', 'dog']):
    folder_path = os.path.join(dataset_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            feature = extract_features(image_path)
            features.append(feature)
            labels.append(label)

# Convert the features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Combine the features and labels into a single numpy array
data = np.hstack((features, np.expand_dims(labels, axis=1)))

# Create a pandas DataFrame from the numpy array
columns = ['feature' + str(i) for i in range(features.shape[1])]
columns.append('label')
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/resnet50v2_dogs_cats_features.csv', index=False)


1/1 [==============================] - 0s 33ms/step


In [21]:
import pandas as pd
ds = pd.read_csv('/content/drive/MyDrive/resnet50v2_dogs_cats_features.csv')

In [22]:
ds

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature100343,feature100344,feature100345,feature100346,feature100347,feature100348,feature100349,feature100350,feature100351,label
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
2,0.0,0.000000,0.331191,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.348376,0.0,0.000000,0.000000,0.0,0.0
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.040862,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.074935,0.0,1.0
997,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.0
998,0.0,0.398834,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.740898,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.0
999,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.0


In [23]:

from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score




x_train ,x_test , y_train, y_test = train_test_split(ds.drop('label', axis=1), ds['label'], test_size=0.3, random_state=42)

# training

param_grid = {'C': [0.1, 1, 10]}
svm = SVC(kernel='linear')

grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(x_train, y_train)




GridSearchCV(cv=5, estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 1, 10]})

In [24]:
results = grid_search.cv_results_
for i in range(len(results['params'])):
    print("Parameters:", results['params'][i])
    print("score:", results['mean_test_score'][i])
    print()

Parameters: {'C': 0.1}
score: 0.99

Parameters: {'C': 1}
score: 0.99

Parameters: {'C': 10}
score: 0.99



In [25]:
y_pred = grid_search.best_estimator_.predict(x_test)
print("Best hyperparameters:", grid_search.best_estimator_)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Best hyperparameters: SVC(C=0.1, kernel='linear')
Accuracy: 0.9800664451827242


In [26]:
from sklearn.metrics import confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test,y_pred)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")
print(cm)


Accuracy: 0.9800664451827242, Precision: 0.9801324503311258, Recall: 0.9801324503311258, F1-score: 0.9801324503311258
[[147   3]
 [  3 148]]


In [14]:
from keras.applications.xception import Xception,preprocess_input
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

dataset_path = '/content/drive/MyDrive/cats_dogs'

output_size = (224, 224)

# Load the ResNet50V2 CNN
model = Xception(weights='imagenet', include_top=False)

#function to extract features from an image
def extract_features(image_path):
    img = load_img(image_path, target_size=output_size)
    img = img_to_array(img)
    img = preprocess_input(img)
    features = model.predict(np.array([img]))
    features = features.flatten()
    return features

#extracting  the features of each image in the directories
features = []
labels = []
for label, folder in enumerate(['cat', 'dog']):
    folder_path = os.path.join(dataset_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            feature = extract_features(image_path)
            features.append(feature)
            labels.append(label)

# Convert the features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Combine the features and labels into a single numpy array
data = np.hstack((features, np.expand_dims(labels, axis=1)))

# Create a pandas DataFrame from the numpy array
columns = ['feature' + str(i) for i in range(features.shape[1])]
columns.append('label')
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Xception_dogs_cats_features.csv', index=False)

1/1 [==============================] - 0s 35ms/step


In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Xception_dogs_cats_features.csv')

In [5]:
df

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature100343,feature100344,feature100345,feature100346,feature100347,feature100348,feature100349,feature100350,feature100351,label
0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.218447,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.173255,0.000000,0.0
2,0.000000,0.015175,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.103632,0.000000,0.387691,0.000000,0.000000,0.432244,0.0
3,0.112292,0.000000,0.000000,0.0,0.190547,0.116967,0.0,0.0,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.191433,0.177537,0.000000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.106756,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,...,0.080539,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
997,0.000000,0.000000,0.086736,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,...,0.000000,0.000000,0.125016,0.372686,0.000000,0.061260,0.587303,0.000000,0.000000,1.0
998,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.40525,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
999,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,...,0.319566,0.000000,0.000000,0.070636,0.000000,0.000000,0.000000,0.000000,0.018510,1.0


In [7]:
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
x_train2 ,x_test2 , y_train2, y_test2 = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.3, random_state=42)


param_grid = {'C': [0.1, 1, 10]}
svm = SVC(kernel='linear')

grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(x_train2, y_train2)

GridSearchCV(cv=5, estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 1, 10]})

In [8]:
results2 = grid_search.cv_results_
for i in range(len(results2['params'])):
    print("Parameters:", results2['params'][i])
    print("score:", results2['mean_test_score'][i])
    print()
y_pred2 = grid_search.best_estimator_.predict(x_test2)
print("Best hyperparameters:", grid_search.best_estimator_)
accuracy = accuracy_score(y_test2, y_pred2)
print("Accuracy:", accuracy)

Parameters: {'C': 0.1}
score: 0.9828571428571429

Parameters: {'C': 1}
score: 0.9828571428571429

Parameters: {'C': 10}
score: 0.9828571428571429

Best hyperparameters: SVC(C=0.1, kernel='linear')
Accuracy: 0.9833887043189369


In [10]:
from sklearn.metrics import confusion_matrix
accuracy2 = accuracy_score(y_test2, y_pred2)
precision2 = precision_score(y_test2, y_pred2)
recall2 = recall_score(y_test2, y_pred2)
f12 = f1_score(y_test2, y_pred2)
cm2= confusion_matrix(y_test2,y_pred2)
print(f"Accuracy: {accuracy2}, Precision: {precision2}, Recall: {recall2}, F1-score: {f12}")
print(cm2)

Accuracy: 0.9833887043189369, Precision: 0.974025974025974, Recall: 0.9933774834437086, F1-score: 0.9836065573770492
[[146   4]
 [  1 150]]


In [24]:
from keras.applications.vgg16 import VGG16 ,preprocess_input
output_size = (224, 224)

# Load the VGG16 CNN
model = VGG16(weights='imagenet', include_top=False)

#function to extract features from an image
def extract_features(image_path):
    img = load_img(image_path, target_size=output_size)
    img = img_to_array(img)
    img = preprocess_input(img)
    features = model.predict(np.array([img]))
    features = features.flatten()
    return features

#dataset directory path
dataset_path = '/content/drive/MyDrive/cats_dogs'

#extracting the features of each image in the directories
features = []
labels = []
for label, folder in enumerate(['cat', 'dog']):
    folder_path = os.path.join(dataset_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            feature = extract_features(image_path)
            features.append(feature)
            labels.append(label)

# Convert the features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Combine the features and labels into a single numpy array
data = np.hstack((features, np.expand_dims(labels, axis=1)))

# Create a pandas DataFrame from the numpy array
columns = ['feature' + str(i) for i in range(features.shape[1])]
columns.append('label')
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/vgg16_dogs_cats_features.csv', index=False)

1/1 [==============================] - 0s 22ms/step


In [11]:
ds3 = pd.read_csv('/content/drive/MyDrive/vgg16_dogs_cats_features.csv')


In [12]:
ds3

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature25079,feature25080,feature25081,feature25082,feature25083,feature25084,feature25085,feature25086,feature25087,label
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,17.492468,4.189556,0.00000,3.081116,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,28.442377,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,21.516972,0.0,0.0
3,0.0,0.0,0.0,2.386773,0.000000,0.0,0.0,0.000000,28.427299,0.000000,...,10.161475,0.0,0.000000,0.000000,0.000000,0.000000,23.04542,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.000000,2.967393,0.0,0.0,0.000000,0.000000,0.000000,...,9.490526,0.0,0.000000,0.000000,0.000000,8.976587,0.00000,17.543234,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,4.433247,0.0,0.000000,0.000000,0.000000,4.911452,0.00000,0.000000,0.0,1.0
997,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,17.930099,0.000000,0.000000,0.00000,8.076289,0.0,1.0
998,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,3.792549,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,1.0
999,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,11.890043,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,1.0


In [13]:
x_train3 ,x_test3 , y_train3, y_test3 = train_test_split(ds3.drop('label', axis=1), ds3['label'], test_size=0.3, random_state=42)


param_grid = {'C': [0.1, 1, 10]}
svm = SVC(kernel='linear')

grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(x_train3, y_train3)

GridSearchCV(cv=5, estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 1, 10]})

In [14]:
results3 = grid_search.cv_results_
for i in range(len(results3['params'])):
    print("Parameters:", results3['params'][i])
    print("score:", results3['mean_test_score'][i])
    print()
y_pred3 = grid_search.best_estimator_.predict(x_test3)
print("Best hyperparameters:", grid_search.best_estimator_)
accuracy = accuracy_score(y_test3, y_pred3)
print("Accuracy:", accuracy)

Parameters: {'C': 0.1}
score: 0.9742857142857144

Parameters: {'C': 1}
score: 0.9742857142857144

Parameters: {'C': 10}
score: 0.9742857142857144

Best hyperparameters: SVC(C=0.1, kernel='linear')
Accuracy: 0.9867109634551495


In [15]:
accuracy3 = accuracy_score(y_test3, y_pred3)
precision3 = precision_score(y_test3, y_pred3)
recall3 = recall_score(y_test3, y_pred3)
f13 = f1_score(y_test3, y_pred3)
cm3= confusion_matrix(y_test3,y_pred3)
print(f"Accuracy: {accuracy3}, Precision: {precision3}, Recall: {recall3}, F1-score: {f13}")
print(cm3)

Accuracy: 0.9867109634551495, Precision: 1.0, Recall: 0.9735099337748344, F1-score: 0.9865771812080537
[[150   0]
 [  4 147]]


In [6]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.densenet  import DenseNet121, preprocess_input


dataset_path = '/content/drive/MyDrive/cats_dogs'

#the output size
output_size = (224, 224)

# Load the NASNetLarge CNN
model = DenseNet121(weights='imagenet', include_top=False)

#function to extract features from an image

def extract_features(image_path):
    img = load_img(image_path, target_size=output_size)
    img = img_to_array(img)
    img = preprocess_input(img)
    features = model.predict(np.array([img]))
    features = features.flatten()
    return features

#extracting  the features of each image in the directories
features = []
labels = []
for label, folder in enumerate(['cat', 'dog']):
    folder_path = os.path.join(dataset_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            feature = extract_features(image_path)
            features.append(feature)
            labels.append(label)

# Convert the features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Combine the features and labels into a single numpy array
data = np.hstack((features, np.expand_dims(labels, axis=1)))

# Create a pandas DataFrame from the numpy array
columns = ['feature' + str(i) for i in range(features.shape[1])]
columns.append('label')
df = pd.DataFrame(data, columns=columns)





1/1 [==============================] - 0s 27ms/step


In [11]:
df.to_csv('/content/drive/MyDrive/DenseNet121_dogs_cats_features.csv', index=False)

In [13]:

ds4 = pd.read_csv('/content/drive/MyDrive/DenseNet121_dogs_cats_features.csv')

In [14]:
ds4

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature50167,feature50168,feature50169,feature50170,feature50171,feature50172,feature50173,feature50174,feature50175,label
0,0.000000,0.000000,0.013717,0.000000,0.000000,1.089234,0.000000,0.000000,0.000000,0.0,...,0.000000,2.646369,0.928439,0.000000,0.000000,0.518841,0.000000,1.641307,0.226984,0.0
1,0.000000,0.001325,0.000000,0.000000,0.195024,0.091578,0.000000,0.000469,0.255104,0.0,...,0.000000,3.363138,0.919930,1.148160,0.319106,3.864907,0.000000,0.857516,0.000000,0.0
2,0.000106,0.016993,0.000000,0.000000,0.411349,1.263932,0.000472,0.000000,0.333210,0.0,...,0.000000,2.801254,0.000000,2.017175,0.000000,0.190167,0.000000,0.000000,0.000000,0.0
3,0.000000,0.034970,0.000000,0.006575,0.635208,3.996985,0.001480,0.000924,0.591249,0.0,...,0.633886,0.162058,0.000000,0.000000,0.000000,0.000000,0.000000,0.325105,0.000000,0.0
4,0.000000,0.000000,0.014504,0.005227,0.126005,5.410447,0.001084,0.001154,0.038453,0.0,...,0.432450,1.598309,0.000000,0.000000,2.985904,0.000000,1.509677,0.110338,0.678466,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.000000,0.001513,0.000070,0.000862,0.075396,0.877240,0.000000,0.000000,0.992956,0.0,...,0.000000,2.000149,0.391780,0.000000,0.000000,0.294162,0.694846,0.000000,0.000000,1.0
997,0.000000,0.008953,0.000000,0.000000,0.000000,0.471295,0.000000,0.000000,0.000000,0.0,...,0.213886,1.269320,0.000000,0.600158,1.668266,0.000000,1.230224,0.000000,0.396562,1.0
998,0.000000,0.000000,0.000000,0.003315,0.000000,0.000000,0.000000,0.000000,0.054208,0.0,...,0.047554,1.136982,0.000000,0.000000,0.770228,0.000000,0.097945,0.000000,0.000000,1.0
999,0.000874,0.006652,0.004818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,1.710137,2.444767,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.652089,1.0


In [16]:
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
x_train4 ,x_test4 , y_train4, y_test4 = train_test_split(ds4.drop('label', axis=1), ds4['label'], test_size=0.3, random_state=42)


param_grid = {'C': [0.1, 1, 10]}
svm = SVC(kernel='linear')

grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(x_train4, y_train4)

GridSearchCV(cv=5, estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 1, 10]})

In [17]:
results4 = grid_search.cv_results_
for i in range(len(results4['params'])):
    print("Parameters:", results4['params'][i])
    print("score:", results4['mean_test_score'][i])
    print()
y_pred4 = grid_search.best_estimator_.predict(x_test4)
print("Best hyperparameters:", grid_search.best_estimator_)
accuracy = accuracy_score(y_test4, y_pred4)
print("Accuracy:", accuracy)

Parameters: {'C': 0.1}
score: 0.9857142857142858

Parameters: {'C': 1}
score: 0.9857142857142858

Parameters: {'C': 10}
score: 0.9857142857142858

Best hyperparameters: SVC(C=0.1, kernel='linear')
Accuracy: 0.9867109634551495


In [20]:
from sklearn.metrics import confusion_matrix
accuracy4 = accuracy_score(y_test4, y_pred4)
precision4 = precision_score(y_test4, y_pred4)
recall4 = recall_score(y_test4, y_pred4)
f14 = f1_score(y_test4, y_pred4)
cm4= confusion_matrix(y_test4,y_pred4)
print(f"Accuracy: {accuracy4}, Precision: {precision4}, Recall: {recall4}, F1-score: {f14}")
print(cm4)

Accuracy: 0.9867109634551495, Precision: 0.9867549668874173, Recall: 0.9867549668874173, F1-score: 0.9867549668874173
[[148   2]
 [  2 149]]
